# From csv to conll / jsonlines

Before run the eval.ipynb, we need to get individual conll files for evaulation
1. Run "Prepare"
2. Run "Generate individual conll files"

## Prepare

In [21]:
import sys
sys.path.append("../../src")
sys.path.append("../../../../git_clone_repos/fast-coref/src/")

import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import json

from IPython.display import display, HTML
# display(HTML(df.to_html()))

from concurrent.futures import ProcessPoolExecutor, as_completed
from multiprocessing import Event
from common_utils.data_loader_utils import load_mimic_cxr_bySection
from common_utils.coref_utils import resolve_mention_and_group_num
from common_utils.file_checker import FileChecker
from common_utils.common_utils import check_and_create_dirs, check_and_remove_dirs
from data_preprocessing import mimic_cxr_csv2conll, mimic_cxr_conll2jsonlines

FILE_CHECKER = FileChecker()
START_EVENT = Event()


In [19]:
from hydra import compose, initialize
from omegaconf import OmegaConf

config = None
with initialize(version_base=None, config_path="../config", job_name="coreference_resolution"):
        config = compose(config_name="coreference_resolution", overrides=["+coreference_resolution/data_preprocessing@_global_=mimic_cxr"])

## Generate individual conll files

In [22]:
# Source input dir (csv)
config.input_gt.base_dir = "../../output/mimic_cxr/manual_test_set/round1x2"
# Target output dir (conll)
config.temp_gt.base_dir = "../../output/mimic_cxr/coref/individual_conll_ground_truth/round1x2"
config.temp_gt.force_run = False # Force to delete and recreate


In [23]:
check_and_remove_dirs(config.temp_gt.base_dir, config.temp_gt.force_run)
if os.path.exists(config.temp_gt.base_dir):
    print("Individual test conll files found and will be reused.")
else:
    log_out = mimic_cxr_csv2conll.prepare_conll(config, config.input_gt, config.temp_gt)
    with open(config.output.run_statistic, "a", encoding="UTF-8") as f:
        f.write(f"Source: {config.temp_gt.base_dir} \n")
        f.write(json.dumps(log_out, indent=2))
        f.write("\n")

100%|██████████| 100/100 [00:03<00:00, 30.45it/s]
